In [ ]:
#pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [1]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

In [2]:
os.environ['LANGCHAIN_PROJECT']

'chat_mypdf'

Document Loader

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf")

docs = loader.load()

In [ ]:
doc = docs[0]
#print(doc.page_content)

In [22]:
import os

pdfs=[]

for root, dir, files in  os.walk('rag-dataset'):
    #print(root,dir,files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [24]:
docs=[]
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

In [26]:
len(docs)

64

##Document Chunking

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks=text_splitter.split_documents(docs)

In [29]:
len(docs), len(chunks)

(64, 311)

In [30]:
len(docs[0].page_content), len(chunks[0].page_content)

(4340, 981)

In [32]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

(969, 294)

###Document Vector Embedding 

In [33]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [34]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")

In [36]:
len(single_vector)

768

In [39]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [40]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [42]:
len(chunks)

311

In [46]:
ids = vector_store.add_documents(documents=chunks)

In [49]:
vector_store.index_to_docstore_id
len(ids)

311

In [52]:
# # store vector database
# db_name = "health_supplements"
# vector_store.save_local(db_name)

# # load vector database
# new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
# len(new_vector_store.index_to_docstore_id)

###Retreival

In [53]:
question = "what is used to gain muscle mass?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [54]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 100, 'lambda_mult': 1})

In [ ]:
docs = retriever.invoke(question)

# for doc in docs:
#     print(doc.page_content)
#     print("\n\n")

acids than traditional protein sources. Its numerous benefits have made it a popular choice
for snacks and drinks among consumers [3]. Another widely embraced supplement is
caffeine, which is found in many sports and food supplements. Caffeine reduces perceived
effort, minimizes fatigue and pain, and proves to be effective for endurance and high-
intensity activities, which is the choice of consumers [4].
Creatine monohydrate is another well-known supplement used to gain muscle mass
and support performance and recovery. It is known not to increase fat mass and remains
effective even when taken in recommended doses [5]. Despite its popularity in the fitness
Foods 2024, 13, 1424. https://doi.org/10.3390/foods13091424
https://www.mdpi.com/journal/foods



and strength gain among men. We detected more prevalent protein and creatine supplementation
among younger compared to older ﬁtness center users, whereas the opposite was found for vitamin
supplementation. Other authors made similar obse

In [58]:
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
question = "what are the benefits of supplements?"
#question = "what are the benefits of BCAA supplements?"
docs = retriever.invoke(question)

### RAG with LLAMA 3.2 on OLLAMA

In [59]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import ChatOllama

In [60]:
model = ChatOllama(model="llama3.2:1b", base_url="http://localhost:11434")

model.invoke("hi")

AIMessage(content='How can I help you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2024-12-21T01:24:00.4590967Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1924638300, 'load_duration': 1712380900, 'prompt_eval_count': 26, 'prompt_eval_duration': 125000000, 'eval_count': 8, 'eval_duration': 85000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9a85bc9f-c292-4cea-aaf3-5cf34222e4de-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [61]:
prompt = hub.pull("rlm/rag-prompt")

In [62]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [64]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [65]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.\n    If you don't know the answer, just say that you don't know.\n    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.\n    Question: {question} \n    Context: {context} \n    Answer:\n"), additional_kwargs={})])

In [67]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

#print(format_docs(docs))

In [68]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [71]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"

question = "what is used to increase mass of the Earth?"

output = rag_chain.invoke(question)
print(output)

Here are the answers to your question in bullet points:

* The component of Geranium plants that is not directly related to increasing mass of the Earth (Geranium plants) includes: 
  • Extracts containing geranium (71)
  • Other botanical compounds, e.g. caffeine, Bauhinia purpurea, Bacopa monniera, Cirsium oligophyllum, and rauwolscine (Yohimbe) extract.
* DMAA (1,3-dimethylamylamine), a synthetic ingredient found in supplements like OxyELITE Pro, is not directly related to increasing mass of the Earth.

However, I can provide information on how substances might affect human weight and mass in general:

* Some ingredients that may contribute to weight loss or reduced appetite include:
  • Chromium: may help regulate blood sugar levels and promote weight loss
  • Green tea: contains catechins that may aid in metabolism and fat burning
  • Garcinia cambogia: extracts contain hydroxycitric acid, which may inhibit the production of fat during weight loss
* Other substances mentioned that